# Work in progress

Need to fix the clustering of the model

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import zipfile

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [2]:
zf = zipfile.ZipFile('titanic.zip')

In [3]:


if os.path.exists('prediction.csv'):
    os.remove('prediction.csv')
else:
    print('file not here')


file not here


In [4]:
df_test = pd.read_csv(zf.open('test.csv'))
df_train = pd.read_csv(zf.open('train.csv'))
df_count = df_test['PassengerId']

df_new = pd.get_dummies(df_train, dtype = float)
df_new.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,"Name_Abbing, Mr. Anthony","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.001122,0.001122,0.001122,...,0.002245,0.003367,0.003367,0.001122,0.002245,0.004489,0.001122,0.188552,0.086420,0.722783
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,0.033501,0.033501,0.033501,...,0.047351,0.057961,0.057961,0.033501,0.047351,0.066890,0.033501,0.391372,0.281141,0.447876
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df_kmeans = pd.get_dummies(df_train, drop_first=True, dtype = float)

values = {"Age": (df_kmeans["Age"].mean())}
df_kmeans = df_kmeans.fillna(value = values)

#df_train = df_train.dropna()
df_kmeans = df_kmeans.drop(["PassengerId", "Survived"], axis=1)
df_kmeans.describe()

,Pclass,Age,SibSp,Parch,Fare,"Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)","Name_Abelson, Mr. Samuel","Name_Abelson, Mrs. Samuel (Hannah Wizosky)","Name_Adahl, Mr. Mauritz Nils Martin",...,Cabin_F G63,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_Q,Embarked_S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208,0.001122,0.001122,0.001122,0.001122,0.001122,...,0.001122,0.002245,0.003367,0.003367,0.001122,0.002245,0.004489,0.001122,0.086420,0.722783
std,0.836071,13.002015,1.102743,0.806057,49.693429,0.033501,0.033501,0.033501,0.033501,0.033501,...,0.033501,0.047351,0.057961,0.057961,0.033501,0.047351,0.066890,0.033501,0.281141,0.447876
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,29.699118,0.000000,0.000000,14.454200,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,3.000000,35.000000,1.000000,0.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
##Doing K-Means Clustering on the Train Data

scaler = StandardScaler()
scaled_df_kmeans = scaler.fit_transform(df_kmeans)
kmeans_model = KMeans(n_clusters=4)
clusters = kmeans_model.fit_predict(df_kmeans)
df_kmeans.insert(df_kmeans.columns.get_loc("Age"), "Cluster", clusters)
df_kmeans.head(10)

c:\Users\heroa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,Pclass,Cluster,Age,SibSp,Parch,Fare,"Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)","Name_Abelson, Mr. Samuel","Name_Abelson, Mrs. Samuel (Hannah Wizosky)",...,Cabin_F G63,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_Q,Embarked_S
0,3,0,22.000000,1,0,7.2500,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,2,38.000000,1,0,71.2833,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,26.000000,0,0,7.9250,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,2,35.000000,1,0,53.1000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,0,35.000000,0,0,8.0500,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,3,0,29.699118,0,0,8.4583,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,1,2,54.000000,0,0,51.8625,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,3,0,2.000000,3,1,21.0750,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,3,0,27.000000,0,2,11.1333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,2,0,14.000000,1,0,30.0708,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_kmeans_final = df_kmeans[["Pclass", "Cluster", "Age", "SibSp", "Parch", "Sex_male","Embarked_Q", "Embarked_S"]].copy()


#notes, had fare, taking it out. too strong

In [8]:
x = df_kmeans_final

y = df_train["Survived"]

,Pclass,Cluster,Age,SibSp,Parch,Sex_male,Embarked_Q,Embarked_S
0,3,0,22.000000,1,0,1.0,0.0,1.0
1,1,2,38.000000,1,0,0.0,0.0,0.0
2,3,0,26.000000,0,0,0.0,0.0,1.0
3,1,2,35.000000,1,0,0.0,0.0,1.0
4,3,0,35.000000,0,0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
886,2,0,27.000000,0,0,1.0,0.0,1.0
887,1,0,19.000000,0,0,0.0,0.0,1.0
888,3,0,29.699118,1,2,0.0,0.0,1.0
889,1,0,26.000000,0,0,1.0,0.0,0.0


In [10]:
from sklearn.metrics import mean_squared_error
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 45)

rf_model = RandomForestClassifier(n_estimators=100, max_features="sqrt", random_state=45)
rf_model.fit(x_train, y_train)

print(mean_squared_error(y_test, y_test,squared=False))
print(rf_model.score(x_train,y_train))

0.0
0.9396067415730337


In [ ]:
predictions = rf_model.predict(x_test)


print (predictions/(y_test))

In [119]:
importances = rf_model.feature_importances_
columns = x.columns
i=0

while i <len(columns):
    print (f" The importance of feature '{columns[i]}' is {round(importances[i]*100, 2)}%.")
    i += 1

 The importance of feature 'Pclass' is 10.64%.
 The importance of feature 'Cluster' is 5.66%.
 The importance of feature 'Age' is 39.23%.
 The importance of feature 'SibSp' is 6.94%.
 The importance of feature 'Parch' is 5.2%.
 The importance of feature 'Sex_male' is 27.95%.
 The importance of feature 'Embarked_Q' is 1.85%.
 The importance of feature 'Embarked_S' is 2.54%.


In [120]:
df_kmeans_test = pd.get_dummies(df_test, drop_first=True, dtype = float)
#df_kmeans_test = df_kmeans_test.dropna()
df_kmeans_test = df_kmeans_test.drop(["PassengerId"], axis=1)
df_kmeans_test.describe()
print(df_kmeans_test["Age"])

0      34.5
1      47.0
2      62.0
3      27.0
4      22.0
       ... 
413     NaN
414    39.0
415    38.5
416     NaN
417     NaN
Name: Age, Length: 418, dtype: float64


In [121]:
df_kmeans_test[df_kmeans_test['Age'].isna()]

values = {"Age": (df_kmeans_test["Age"].mean())}
df_kmeans_test = df_kmeans_test.fillna(value = values)
df_kmeans_test.describe()

valueFare = {"Fare": (df_kmeans_test["Fare"].mean())}
df_kmeans_test = df_kmeans_test.fillna(value = valueFare)

In [122]:
##Doing K-Means Clustering on the Test Data

scaler = StandardScaler()
scaled_df_kmeans_test = scaler.fit_transform(df_kmeans_test)
kmeans_model_test = KMeans(n_clusters=4)
clusters = kmeans_model_test.fit_predict(df_kmeans_test)
df_kmeans_test.insert(df_kmeans_test.columns.get_loc("Age"), "Cluster", clusters)
df_kmeans_test.head(10)

c:\Users\heroa\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,Pclass,Cluster,Age,SibSp,Parch,Fare,"Name_Abelseth, Miss. Karen Marie","Name_Abelseth, Mr. Olaus Jorgensen","Name_Abrahamsson, Mr. Abraham August Johannes","Name_Abrahim, Mrs. Joseph (Sophie Halaut Easu)",...,Cabin_F,Cabin_F E46,Cabin_F E57,Cabin_F G63,Cabin_F2,Cabin_F33,Cabin_F4,Cabin_G6,Embarked_Q,Embarked_S
0,3,0,34.5,0,0,7.8292,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3,0,47.0,1,0,7.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,0,62.0,0,0,9.6875,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,0,27.0,0,0,8.6625,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,0,22.0,1,1,12.2875,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,3,0,14.0,0,0,9.2250,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,3,0,30.0,0,0,7.6292,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,2,0,26.0,1,1,29.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,3,0,18.0,0,0,7.2292,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,3,0,21.0,2,0,24.1500,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [123]:
df_test_final = df_kmeans_test[["Pclass", "Cluster", "Age", "SibSp", "Parch", "Sex_male","Embarked_Q", "Embarked_S"]].copy()

In [124]:
test_set_P = rf_model.predict(df_test_final)
print (test_set_P)

[0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1 0
 1 0 1 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0
 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 0
 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0
 0 0 1 0 1 1 0 1 0 0 0]


In [125]:
print(df_test.describe())
print(len(test_set_P))


       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  332.000000  418.000000  418.000000  417.000000
mean   1100.500000    2.265550   30.272590    0.447368    0.392344   35.627188
std     120.810458    0.841838   14.181209    0.896760    0.981429   55.907576
min     892.000000    1.000000    0.170000    0.000000    0.000000    0.000000
25%     996.250000    1.000000   21.000000    0.000000    0.000000    7.895800
50%    1100.500000    3.000000   27.000000    0.000000    0.000000   14.454200
75%    1204.750000    3.000000   39.000000    1.000000    0.000000   31.500000
max    1309.000000    3.000000   76.000000    8.000000    9.000000  512.329200
418


In [132]:
kaggle_Predictions = pd.DataFrame({
    "PassengerId":df_count,
    "Survived": test_set_P.astype(float)
})
predictions = pd.DataFrame({
    "PassengerId":df_count,
})

In [127]:
kaggle_Predictions.to_csv('RandomForestPredictionKMeanswithoutfareadjustedage.csv', index=False)

In [133]:
predictions.to_csv('test.csv', index = False)